# Data Visualization Project

## Airplane Crashes Since 1908

***

Authors **Catarina Lourenço**

Student Numbers **C17709355**

Module **Big Data Concepts**

Lecturer **Abhishek Kaushik**

Course-Year **DT302-2 Business Analytics**

Submitted **01-04-2019**

***


## Summary
Data Visualization Project using Python and other tools, exploring [open data about aviation crashes](https://www.kaggle.com/saurograndi/airplane-crashes-since-1908), sourced from Kaggle.

*Python installation version:* anaconda3-5.1.0

*Additional packages:* geopy

*Other tools:* Google Fusion Tables, Excel


### Open data source selected: 
[Aviation crashes dataset](https://www.kaggle.com/saurograndi/airplane-crashes-since-1908)

#### Database Format
| Column Name | Summary |
|---------------|------------------------------------------------------|
| Date:         | Date of accident,  in the format - January 01, 2001 |
| Time:         | Local time, in 24 hr. format unless otherwise specified |
| Airline/Op:   | Airline or operator of the aircraft |
| Flight #:     | Flight number assigned by the aircraft operator |
| Route:        | Complete or partial route flown prior to the accident |
| AC Type:      | Aircraft type |
| Reg:          | ICAO registration of the aircraft |
| cn / ln:      | Construction or serial number / Line or fuselage number |
| Aboard:	    | Total aboard (passengers / crew) |
| Fatalities:	| Total fatalities aboard (passengers / crew) |
| Ground:	    | Total killed on the ground |
| Summary:	    | Brief description of the accident and cause if known |

### Final dataset used for analysis 

The original dataset wasn't used *as-is*, as we had to change some of its structure and contents in order to prepare it for the kind of analysis we had proposed ourselves to perform. 

**Data cleaning** involved mainly enforcing a coherent format within each column, decluttering text from words like "near" to keep only the relevant information, replacing NaN and strings by the correct numeric literal.

**Additions** Two columns for GPS coordinates, Latitude and Longitude. This was done using the geopy library (explain further in another section...) to scrap data from BingMaps. The script used can be found [here](add_coordinates.py).

**Deletions** The columns *Flight #*, *Route*, *Registration*, *cn/In* were dropped as we didn't require them for our analysis. Additionally, three rows for which the entries couldn't be verified were also dropped.

[This is the dataset](dataset/clean_v1_Airplane_Crashes_and_Fatalities_Since_1908.csv) that we used to start cleaning and preparing the data using pandas - See Part 1. Data Preparation.

[This is the final dataset](dataset/dataset_final.csv) that we used to proceed with analysis - See Part 2. Exploratory Data Analysis.

### Questions to be answered: 

**Descriptive:**
* Fatalities before and after hitting the ground
* Where do most planes crash (by time period)? Heat map with locations - period of 5-10 years? - change over time - animation/several time periods (WW2)?
* Separate into sections? Location by continent, night/day flight, type of aircraft etc
* Is there a specific make/model that appears more frequently than others
* Are they mainly passenger airplanes/cargo airplanes
* Does the age of the plane matter,ie, an older/newer plane regarding crashes

**Predictive:**
* When/where should we expect the next crash to happen? With how many fatalities?

**Text analysis:**
* Text analysis: find causes in description text
* (Advanced) Text analysis - sentiment analysis

## 1. Data Preparation

---
Cleaning, completing, dropping, converting

### 1.1 Inspecting the Data

After the initial data read and conversion to a dataframe, we were able to run the ```info()``` method, which showed us there were several missing values - we are able to tell this by the count of values on each column against the total of 5268 entries:


In [3]:
import pandas as pd

file_path = "dataset/clean_v1_Airplane_Crashes_and_Fatalities_Since_1908.csv"
dframe = pd.read_csv(file_path)

# get information about data entries by column:
dframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 11 columns):
Date          5268 non-null object
Time          3049 non-null object
Location      5248 non-null object
Latitude      5240 non-null float64
Longitude     5240 non-null float64
Operator      5250 non-null object
Type          5241 non-null object
Aboard        5246 non-null float64
Fatalities    5256 non-null float64
Ground        5246 non-null float64
Summary       4878 non-null object
dtypes: float64(5), object(6)
memory usage: 452.8+ KB


* Date          5268 
* Time          3049  *-> missing 2219 values*
* Location      5248  *-> missing 20 values*
* Latitude      5240  *-> missing 28 values (generated from Location, missing 8)*
* Longitude     5240  *-> missing 28 values (generated from Location, missing 8)*
* Operator      5250  *-> missing 18 values*
* Type          5241  *-> missing 19 values*
* Aboard        5246  *-> missing 22 values*
* Fatalities    5256  *-> missing 12 values*
* Ground        5246  *-> missing 22 values*
* Summary       4878  *-> missing 390 values*


In [4]:
dframe.describe()

,Latitude,Longitude,Aboard,Fatalities,Ground
count,5240.000000,5240.000000,5246.000000,5256.000000,5246.000000
mean,27.010367,-16.061076,27.554518,20.068303,1.608845
std,24.570333,83.419121,43.076711,33.199952,53.987827
min,-77.529716,-176.669861,0.000000,0.000000,0.000000
25%,10.944428,-82.810095,5.000000,3.000000,0.000000
50%,34.392923,-9.149480,13.000000,9.000000,0.000000
75%,43.801488,37.615021,30.000000,23.000000,0.000000
max,80.449997,178.800476,644.000000,583.000000,2750.000000


Running the ```describe()``` method, we can spot some possible errors in Aboard: the minimum shouldn't be zero, because there are no unmanned flights in this dataset - we'll have to investigate this, as it could be due to missing values or a wrong value. Ground also has a maximum value that is abnormaly high - 2750 people killed on impact. All these warrant inspection.

The fact that there are only 4673 unique summaries of the crash doesn't necessarily point to duplicate entries - it might just be the summary was the same kind of description, such as "Crashed on landing".

### 1.2 Data Cleaning

Before proceeding with inspecting individual rows, changes to datatypes or removing duplicates, we checked for NaN and null values.

> isna(obj)	    Detect missing values for an array-like object.
>
> isnull(obj)	Detect missing values for an array-like object.
>
> notna(obj)	Detect non-missing values for an array-like object.
>
> notnull(obj)	Detect non-missing values for an array-like object.

For that purpose, we created two small methods that check for these column-wise:

```python
#Nan and lower than x values (useful for checking zeros, for instance)

def check_nulls(column, dataframe=dframe):
    column_series = dataframe[column]
    null_column = column_series[(column_series.isna()) | (column_series.isnull())]
    print("No null values in {0} series.".format(column)) if null_column.empty else print(null_column)

def check_lower_than(column, value=1, dataframe=dframe):
    column_series = dataframe[column]
    filtered_column = column_series[(column_series < value)]
    print("No values lower than {1} in {0} series.".format(column, value)) if filtered_column.empty else print(filtered_column)

```

Using these, we could generate small reports that tell us the rows we need to fix (NaN), and possibly investigate to check if the values are correct (zeros). A ```0``` on the Ground column is not "supicious", in fact it is the most common value, as that column indicates fatalities *on impact*. However, a ```0``` on Aboard is definitely a mistake.

***
An example:

>--- ABOARD
>
>No null values in Aboard series.
>
>
>*3307*    0
>
>*3611*   0
>
>Name: Aboard, dtype: int16

***

The process of filling these missing values was slow and laborious, as each entry had to be checked, researched and changed manually. This is documented in detail in the Data Preparation notebook (DataPreparation.ipynb) in annex to this report.

During this process, the fact that many of the entries contained the wrong information became apparent - the wrong date, misspellings, wrong values for Aboard or Fatalities. Nothing too egregious, but it was a reminder that this data was scrapped by someone else, from a source that someone else had put together online, and so its veracity isn't exactly 100%.

Another type of cleaning we performed was on Summary and Location text. For Summaries, we filled empty strings or null values with a placeholder that reads "No details available". For Locations, we identified a few recurring words and typos and corrected them using the ```.str.replace()``` method that pandas has available for ```DataFrame``` objects.

Example:

```python
#check if these words are in the values
df[df.Location.str.contains(r"Near|Off the|Over the")]["Location"]

#to remove/replace those words and then strip trailing whitespaces
df.Location = df.Location.str.replace("Near", "")
df.Location = df.Location.str.replace("Over the", "")
df.Location = df.Location.str.strip()
#replace 
df.Location = df.Location.str.replace("iO", "ic O")
df.Location = df.Location.str.replace("iR", "ic R")
df.Location = df.Location.str.replace("iG", "ic G")

#check
df[df.Location.str.contains(r"i[\\A-Z]")]["Location"] #should return empty list if all these typos were corrected

```

#### 1.2.1 Adding geographic data with GeoPy

The final step in data completion was to run the ```add_coordinates.py``` script on a .csv file with all the location names cleaned up, which returned a second .csv file with the appended data for latitude and longitude.
For this script, we used the GeoPy library to handle the API requests to a Geolocation service of our choice - for ease of use and accuracy of results, Bing Maps was selected. 



### 1.3. Type conversions

With all the values in place, we could then proceed to convert them to more useful dtypes. Fatalities, Aboard and Ground refer to a discrete number, so these were converted to ```int```.

```python
df['Fatalities'] = pd.to_numeric(df['Fatalities'], downcast="integer", errors="coerce")

df['Ground'] = pd.to_numeric(df['Ground'], downcast="integer", errors="coerce")

```

The most crucial conversion was Time and Date to Datetime objects, as this allowed us to better manipulate and access these data. With a Datetime object, pands gives us specific .dt methods to use: for instance, to extract the Year, to get comparisons/deltas between dates or times, or to return the day of the week that date was on.

```python
df["Time"] = pd.to_datetime(df["Time"], format='%HH:%MM', errors="ignore")
```


### 1.4 Dropping entries and Series

At the initial stages of getting to know the dataset and planning our analysis, we identified that the columns *Flight #*, *Route*, *Registration*, *cn/In* weren't useful for what we had proposed to look at. These four columns were dropped. Additionally, three rows (364, 423, 768) for which the entries couldn't be verified were also dropped. This was followed by reseting the index for all the dataframe entries.

```python
#dropping and reindexing
df = dframe.drop([364, 423, 768])
df = df.reset_index(drop=True)

#checking duplicates
df_dupes = df.duplicated() #returns a series with bool values
df_dupes[(df_dupes == True)] #filter: if it returns an empty list, there are no duplicate rows... happy days!
```


### 1.5 Data Preparation results

The end result was an almost fully complete dataset - Time, Operator and Type are the only incomplete columns -, with no missing values in the columns that we'll base most of our analysis on - Date, Location, Aboard, Fatalities and Ground. The resulting dataset was exported as .csv to keep as a backup record.

With this, we can move on to the next step, EDA.



___

## 2. Exploratory Data Analysis


### 2.1 




### 2.3 Google Fusion Table

The [table](https://www.google.com/fusiontables/DataSource?docid=1vGwRQjKd9R2-yRNynsV9_Dn2ecNYn4YE7mVPsewZ)



## 3. Predictive Analysis

## Conclusions